In [204]:
import numpy as np
import matplotlib.pyplot as plt


In [205]:
def objective_j(p,std_sample_time):

    threshold = [2,3,1]
    
    rate = 0.3

    def eta(i):
        return np.random.normal(0, np.sqrt(i+1), size = std_sample_time)

    def V():
        return np.random.normal(0, 1, std_sample_time)

    def W():
        return np.random.exponential(rate, std_sample_time)

    def Yi(xi):
        return np.random.uniform(0, xi, std_sample_time)
    
    Yi_values= np.zeros((3,std_sample_time))
    xi_values = np.zeros((3,std_sample_time))
    PiYi = np.zeros(std_sample_time)

    for i in range(3):

        xi_values[i] = (0.6 * V() + 0.8 * eta(i)) / (np.maximum(W(), 1))
        Yi_values[i] = np.where(xi_values[i] >= threshold[i], Yi(xi_values[i]), 0)

        if i != 2:
            PiYi += p[i] * Yi_values[i]
        else:
            PiYi += (1-p[0]-p[1]) * Yi_values[2]
        
    result_std = np.std(PiYi)

    return PiYi[0] / result_std if result_std!=0 else 0

objective_j([0.6,0.4],10)

0.0

In [206]:
def exp_objective_j(p,sample_for_exp,N):
    j_values = np.zeros(sample_for_exp)
    for i in range(sample_for_exp):
        j_values[i] = objective_j(p,N)
    return j_values.mean()

exp_objective_j([0.6,0.4],5000,10)

0.0861980057236467

In [207]:
def estimate_gradient(p, i, rng):
    delta_i = rng.choice((-1, 1), size = 2)
    eta_i = 1/(10 + np.power(i,1/4))

    perturbation_high = p + eta_i * delta_i
    perturbation_low = p - eta_i * delta_i
    obj_perturbation_high = exp_objective_j(perturbation_high, EXP_SAMPLE_TIME, STD_SAMPLE_TIME)
    obj_perturbation_low = exp_objective_j(perturbation_low, EXP_SAMPLE_TIME, STD_SAMPLE_TIME)
    numerator = obj_perturbation_high - obj_perturbation_low
    denominator = 2*eta_i*delta_i
    gradient_estimate = np.divide(numerator, denominator)

    return gradient_estimate

estimate_gradient([0.6,0.4], 1, np.random.default_rng())

array([-4.68412156, -4.68412156])

In [208]:
def project_onto_plane(p):
    
    if 1 >= p[0] >= 0 and 1 >= p[1] >= 0 and 0 <= p[0] + p[1] <= 1:
        return p
    else:
        if p[0] < 0:
            p[0] = 0
        if p[1] < 0:
            p[1] = 0
        if p[0] + p[1] > 1:
            if p[1] - p[0] > 1:
                return [0,1]
            elif p[1] - p[0] < -1:
                return [1,0]
            else:
                tmp0, tmp1 = p[0],p[1]
                p[0] = (tmp0 - tmp1 + 1) / 2
                p[1] = (tmp1 - tmp0 + 1) / 2
    return p

In [209]:
# The SPSA algorithm
def SPSA(SEED, p_0, EPSILON_TYPE, EPSILON_VALUE, NR_ITERATIONS):
    rng = np.random.default_rng(SEED)
    p_set = np.zeros((NR_ITERATIONS+1, 2))  # 2 is the number of elements in each p vector
    gradients = np.zeros((NR_ITERATIONS, 2))
    objective_values = np.zeros(NR_ITERATIONS+1)
    p_set[0] = p_0
    objective_values[0] = exp_objective_j(p_set[0], EXP_SAMPLE_TIME, STD_SAMPLE_TIME)

    for i in range(NR_ITERATIONS):
        g = estimate_gradient(p_set[i], i, rng)
        gradients[i] = g
        if EPSILON_TYPE == 'fixed':
            p_set[i+1] = p_set[i] + EPSILON_VALUE * g
        if EPSILON_TYPE == 'decreasing':
            p_set[i+1] = p_set[i] + 1/(i+1) * g
        p_set[i+1] = project_onto_plane(p_set[i+1])
        objective_values[i+1] = exp_objective_j(p_set[i+1], EXP_SAMPLE_TIME, STD_SAMPLE_TIME)
        
    return p_set, gradients, objective_values

In [210]:
p_0 = np.array([0.37, 0.20])
SEED = 1
EPSILON_TYPE = 'fixed'  # "fixed" or "decreasing"
EPSILON_VALUE = 0.001
NR_ITERATIONS = 2000
EXP_SAMPLE_TIME = 1000 # each round to get expected value
STD_SAMPLE_TIME = 10

In [211]:
def draw_p_graph_modified(thetas,object_values,gradients):

    if thetas.shape[1] == 2:
        #2-dimensional plots.
        fig, axs = plt.subplots(2, 2, figsize=(10,10))

        # Plot the iterate
        axs[0,0].plot(thetas.T[0], color="darkblue", label = r'$p_1$')
        axs[0,0].plot(thetas.T[1], color="deepskyblue", label = r'$p_2$')
        
        p0_sol = np.mean(thetas.T[0][int(0.75*NR_ITERATIONS):])
        p1_sol = np.mean(thetas.T[1][int(0.75*NR_ITERATIONS):])
        
        axs[0,0].axhline(y=p0_sol, color='black', linestyle='--')
        axs[0,0].axhline(y=p1_sol, color='black', linestyle='--')

        print(f"p1: {p0_sol}")
        print(f"p2: {p1_sol}")

        axs[0,0].set_xlabel("Iteration")
        axs[0,0].legend()

        # plot contour
        r = 1
        x = np.linspace(0, r, 5)
        y = np.linspace(0, r, 5)

        X, Y = np.meshgrid(x, y)
        Z = np.array([[ exp_objective_j( [X[i,j], Y[i,j]], EXP_SAMPLE_TIME, STD_SAMPLE_TIME) for j in range(len(x))] for i in range(len(x))])

        CS = axs[0,1].contour(X, Y, Z)
        axs[0,1].clabel(CS, inline=1, fontsize=10)

        axs[0,1].plot(thetas.T[0], thetas.T[1], c="red", linewidth=1)
        axs[0,1].set_xlim(0,r)
        axs[0,1].set_ylim(0,r)
        axs[0,1].set_xlabel(r"$p_1$")
        axs[0,1].set_ylabel(r"$p_2$") 

        # Plot the objective values
        axs[1,0].plot(object_values, color="darkblue")
        axs[1,0].set_xlabel("Iteration")
        result_mean = np.mean(object_values[int(0.75 * NR_ITERATIONS):])
        axs[1, 0].axhline(y=result_mean, color='black', linestyle='--')
        print(f"Result mean of last 25% run: {result_mean}")

        # Plot the estimated gradients
        axs[1,1].plot(gradients.T[0], color="darkblue", label = r'$g^{SPSA}_1$')
        axs[1,1].plot(gradients.T[1], color="deepskyblue", label = r'$g^{SPSA}_2$')
        axs[1,1].set_xlabel("Iteration")
        axs[1,1].set_ylabel(r"$g^{SPSA}$($p_n$)")
        axs[1,1].set_title(r"$g^{SPSA}$($p_n$)")
        axs[1,1].legend()

          # Plot the 3D graph
        fig = plt.figure(figsize=(8, 6))
        ax = fig.add_subplot(111, projection='3d')
        ax.plot(thetas.T[0], thetas.T[1], object_values, 'r')
        ax.set_xlabel(r"$p_1$")
        ax.set_ylabel(r"$p_2$")
        ax.set_zlabel(r"f($J(p)$)")
        ax.set_title("Objective Values in 3D")

        

In [ ]:
p_set,gradients,object_values = SPSA(SEED, p_0, EPSILON_TYPE, EPSILON_VALUE, NR_ITERATIONS)
draw_p_graph_modified(p_set,object_values,gradients)